# Every ML Algorithm Explained and Implemented From Scratch 1: Linear Regression

## Introduction
In this notebook, we look at Linear Regression. Only knowledge of basic linear algebra and probability theory is assumed. 
## Linear Regression - Theory
### Motivation
Suppose we have a data set $x_1,y_1,x_2,y_2...,x_m,y_m$ where $x_i \in \mathbb{R}^d$ is the *predictor* or *independent variable*, and $y\in \mathbb{R}$ is the *response* or *dependent variable*. For example, $x_i$ can be a vector consisting of a patient's weight and resting blood pressure, and $y_i$ might be the patient's blood sugar. Our goal is to solve the *regression problem*: given a (previously unseen) vector $x \in \mathbb{R}^d$, we want to predict the corresponding $y\in \mathbb{R}$ as accurately as possible.

When doing linear regression, we assume that the relationship between the predictor variable and the response variable is *approximately linear* - i.e., that there is a linear map $f:\mathbb{R}^d \rightarrow \mathbb{R}$ such that $y \approx f(x)$. There are several ways in which we can formalize this statement, and we look at some of them in the next sections.

### Gaussian Errors
Let's assume that the relationship between the predictor variable and the response variable is linear plus a Gaussian error with mean zero and constant variance i.e., there is a linear map $f:\mathbb{R}^d \rightarrow \mathbb{R}$ such that for all $i$, $y_i=f(x_i)+\epsilon_i$ where $\epsilon_i \sim N(0,\sigma^2)$ and $\epsilon_i$ is independent of $\epsilon_j$ for all $i\neq j$.

If we knew $f$, the best prediction we could give for $x \in \mathbb{R}^d$ would be $f(x)$. However, since we don't know $f$, we will have to estimate it, and we will do so using a *maximum likelihood* argument.

For a vector $x \in \mathbb{R}^d$, denote by $(x,1)$ the vector obtained from $x$ by appending a 1 (so if, for example, $x=(5,7,3)$ then $(x,1)=(5,7,3,1)$. Note that every linear map $f:\mathbb{R}^d \rightarrow \mathbb{R}$ can be expressed as an inner product in $\mathbb{R}^{d+1}$: we have $f(x)=\theta_f\cdot(x,1)$ for some $\theta_f\in \mathbb{R}^{d+1}$. Thus, to estimate $f$ it is enough to estimate $\theta_f$. 

For a fixed $\theta_f$ and $x_i$, the likelihood function of the value $y_i$ is the likelihood that $\epsilon_i=y_i-\theta_f\cdot(x,1)$. Since $\epsilon_i$ is normally distributed, we can easily compute this likelihood using the Gaussian distribution $n(x)=\frac{1}{\sqrt{2\pi}}exp\{-\frac{x^2}{2\sigma^2}\}$. Combining this with the fact that the $\epsilon_i$'s are pairwise independent, we can compute the joint likelihood function of $\theta_f$:

$$
\mathcal{L}(\theta_f|x_1,y_1,...,x_m,y_m)=\prod_{i=1}^m p(y_i|x_i,\theta_f)=\prod_{i=1}^m n(y_i-\theta_f\cdot(x_i,1))
$$

Taking $\log$, we get:
$$
\log(\mathcal{L}(\theta_f|x_1,y_1,...,x_m,y_m))=\sum_{i=1}^m \frac{1}{2\sigma^2}(y_i-\theta_f\cdot(x_i,1))^2 +const
$$
Now, taking the derivative with respect to $\theta_f$ (note that $\theta_f$ is a vector, so we are computing a gradient), we get:
$$
\frac{\partial\log(\mathcal{L})}{\partial \theta_f}= \frac{1}{2\sigma^2}\sum_{i=1}^m-2[y_i-\theta_f\cdot(x_i,1)]\cdot(x_i,1)
$$
Denoting by $X\in \mathbb{R}^{m\times (d+1)}$ the matrix whose $i$th row is $(x_i,1)$ and by $Y=(y_1,...,y_m)^t \in\mathbb{R}^{m\times 1}$, we can rewrite the above as
$$
\frac{\partial\log(\mathcal{L})}{\partial \theta_f}=-\frac{1}{\sigma^2}[Y^t-\theta_f^tX^t]X=-\frac{1}{\sigma^2}[Y^tX-\theta_f^tX^tX]
$$
Equating to 0, we get that $\log(\mathcal{L})$ is minimized when
$$
\theta_f^t=Y^tX(X^tX)^{-1}
$$
or
$$
\theta_f=(X^tX)^{-1}X^tY
$$
Note $X^tX$ is invertible, since it is positive definite (assuming $m>d+1$ i.e., we have more samples than coordinates, and there are no duplicate data points). 

$\theta_f$ has a geometrical interpretation: recall that $X(X^tX)^{-1}X^t$ is the projection operator onto the subspace spanned by the columns of $X$, so $X(X^tX)^{-1}X^tY=X\theta_f$ is the point in that subspace closest to $Y$, using the Euclidian metric. Consider the system of equations $Y=X\theta$. This system does not have a solution, and $\theta_f$ is the best *approximate* solution, in the sense that $||Y-X\theta_f||^2$ is minimized.

### Implementation
Implementing the linear regression is as simple as inverting and multiplying some matrices. We will start with an example where $d=1$ for illustration.

In [1]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

np.random.seed(seed=42)

x=np.arange(1,10,0.2)
f_x=np.vectorize(lambda x: 3*x+2)(x)
errors=np.random.normal(scale=1.0, size=x.shape)
y=f_x+errors
fig=px.scatter(x=x,y=y)
fig.add_shape(type="line", x0=0, y0=2, x1=10, y1=32)
fig.show()

Here we generated the data according to the formula $y_i=3x_i+2+\epsilon_i$, where $\epsilon_i$ is a Gaussian sample with mean $0$ and variance $1.0$. Let's use the formula we derived to find $\theta_f$:

In [2]:
class LinearRegressor:
    def __init__(self):
        self.theta_f=None
        
    def fit(self, X,y):
        if X.ndim==1:
            X1=np.stack([X,np.ones(shape=X.shape)]).T
        else:
            X1=np.vstack([X.T,np.ones(shape=X[:,0].shape)]).T
        self.theta_f=np.linalg.inv(X1.T@X1)@X1.T@y
        
    def predict(self, X):
        if self.theta_f is None:
            raise Exception("The model has not been fit!")
        if X.ndim==1:
            temp=np.stack([X,np.ones(shape=X.shape)])
        else:
            temp=np.vstack([X.T,np.ones(shape=X[:,0].shape)])
        return self.theta_f@temp


model_1=LinearRegressor()
model_1.fit(x,y)
print(model_1.theta_f)

[2.92651696 2.18056594]


For a given $x$, the linear regression will predict $y=2.93x+2.18$ - not too far from the real function.

### Evaluating the Model: MSE and $R^2$

How do we evaluate how well the linear model fits the data? Two common methods are using the Mean Square Error and the Coefficient of Determination, known as $R^2$.
Recall we have shown
$$
\theta_f=argmin_\theta ||Y-X\theta||^2
$$
It makes sense to use the expression we are trying to minimize as a measure of the error, thus we define 
$$
MSE=\frac{||Y-X\theta_f||^2}{m}=\frac{\sum_{i=1}^m (y_i-(x_i,1)\cdot\theta_f)^2}{m}=\frac{\sum_{i=1}^m (y_i-y'_i)^2}{m}
$$
where $y'_i$ is the prediction the model assigns to $x_i$. We divide by $m$ since we want the mean loss for every sample.

Another way to measure the accuracy of the model is $R^2$. Denote by $SSR$ the sum of square residuals:
$$
SSR=||Y-X\theta_f||^2
$$
Note that $SSR=mMSE$.

Consider a naive model that assigns a constant value to every input. What would be the best constant to assign, in the sense of minimizing $\sum_{i=1}^m (y_i-y'_i)^2$? Clearly, it would be the mean of the samples, $\bar{y}=\frac{\sum_{i=1}^m y_i}{m}$.

The SSR of the model that assigns the value $\bar{y}$ to every model is known as the Total Sum of Squares, or TSS, and is equal to:
$$
TSS=\sum_{i=1}^m (y_i-\bar{y})^2
$$
The ratio $\frac{SSR}{TSS}$ is a measure of how much better our model does than the naive model. We know that $SSR\leq TSS$, since $TSS=||Y-X\theta||^2$ where $\theta=(0,...0,\bar{y})$, and hence $\frac{SSR}{TSS}\leq1$. Subtracting this quantity from $1$, we get $R^2$:
$$
R^2=1-\frac{SSR}{TSS}\leq 1
$$
The closer $R^2$ is to 1, the more accurate the model.

Let's compute the MSE and $R^2$ for the example above:

In [3]:
y_pred=model_1.predict(x)
errors=y-y_pred
ssr=np.linalg.norm(errors)
mse=ssr/len(y_pred)
y_bar=np.mean(y)
tss=np.linalg.norm(y-y_bar*np.ones(shape=y.shape))
print(f"TSS={tss}, SSR={ssr}, MSE={mse}, R^2={1-ssr/tss}")

TSS=51.34804621504298, SSR=6.036488858884913, MSE=0.13414419686410917, R^2=0.882439755670461


Our model has an MSE  of $0.13$ and an $R^2$ of $0.88$. To contrast, let's try fitting a linear regression on two more data sets. In both data sets, the $x$ values will be the same as the original, but now the $y$ values in the second are generated according to $y_i=2x_i^2-5+\epsilon_i$, and in the third according to $y_i=0.5x^3-4$.

In [4]:
#Relabel the original data, for convinience.
x1=x
y1=y
y1_pred=y_pred

#Generate new data
x2=x1.copy()
f_x2=np.vectorize(lambda x: 2*x**2-5)(x2)
errors2=np.random.normal(scale=1.0, size=x2.shape)
y2=f_x2+errors2

x3=x1.copy()
f_x3=np.vectorize(lambda x: 0.5*x**3-4)(x3)
errors3=np.random.normal(scale=1.0, size=x3.shape)
y3=f_x3+errors3

#Train new models
model_2=LinearRegressor()
model_2.fit(x2,y2)
y2_pred=model_2.predict(x2)

model_3=LinearRegressor()
model_3.fit(x3,y3)
y3_pred=model_3.predict(x3)

#Calculate MSE and R^2
errors2=y2-y2_pred
ssr2=np.linalg.norm(errors2)
mse2=ssr2/len(errors2)
y2_bar=np.mean(y2)
tss2=np.linalg.norm(y2-y2_bar*np.ones(shape=y2.shape))

errors3=y3-y3_pred
ssr3=np.linalg.norm(errors3)
mse3=ssr3/len(errors3)
y3_bar=np.mean(y3)
tss3=np.linalg.norm(y3-y3_bar*np.ones(shape=y3.shape))


print(f"Second data set: TSS={tss2}, MSE={mse2}, R^2={1-ssr2/tss2}")
print(f"Third data set: TSS={tss3}, MSE={mse3}, R^2={1-ssr3/tss3}")

Second data set: TSS=385.0603145878776, MSE=1.7913309054713429, R^2=0.7906564564242732
Third data set: TSS=928.2959750960043, MSE=7.321875775739736, R^2=0.6450653468854985


As we can see, the MSE and $R^2$ values indicate that the linear model fits the second data set less well than the first and the third data set less well than the second. Note that the $R^2$ values for both new data sets are still not that bad - both are above $0.6$. This is because this close to $0$, the quadratic and qubic functions are "not that far" from being linear. We will discuss this more later, but first, let's plot all three data sets and models. In the following plot, the dots are the data points, and the lines are corresponding linear models.

In [5]:
fig = make_subplots(rows=1, cols=3)
fig.add_trace(
    go.Scatter(x=x1, y=y1, mode="markers", name="Data set 1"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=x2, y=y2, mode="markers", name="Data set 2"),
    row=1, col=2
)
    
fig.add_trace(
    go.Scatter(x=x3, y=y3, mode="markers", name="Data set 3"),
    row=1, col=3
)

fig.add_shape(type="line",x0=0, y0=np.vdot(model_1.theta_f,np.array([0,1])), x1=10, y1=np.vdot(model_1.theta_f,np.array([10,1])), row=1, col=1, )
fig.add_shape(type="line",x0=0, y0=np.vdot(model_2.theta_f,np.array([0,1])), x1=10, y1=np.vdot(model_2.theta_f,np.array([10,1])), row=1, col=2,name="Data set 2")
fig.add_shape(type="line",x0=0, y0=np.vdot(model_3.theta_f,np.array([0,1])), x1=10, y1=np.vdot(model_3.theta_f,np.array([10,1])), row=1, col=3, name="Data set 3")
fig.update_layout(title_text="Three Linear Models")    
fig.show()

### Notes and final considerations
Some final thoughts and considerations you should bear in mind when using linear regression:
* The linear regression is usually only accurate for values in the range $[\min(x_1,...,x_m), \max(x_1,...,x_m)]$. (As an example, consider the second and third data sets and models above: The lines are fairly close to the data points between 1 and 10, but even the value for 0 is already quite far away.
* The derivation of the linear regression was done on the assumption that the errors $\epsilon_i$ are all mean zero normal with the same variance. When this assumption does not hold, the data is called heteroscedastic, and the model may not be accurate. 

## Regularization - Ridge and LASSO Regression
### Motivation
As we have seen, solving the linear regression problem is equivalent to solving the optimization problem
$$
\min_\theta ||Y-X\theta||^2
$$
There may be two problems we might want to solve:

**Sensitivity to Outliers** - consider the following two data sets. Both are the same as the first data set, with two changes: the variance of the error term was increased to 4. Additionally, the value for $x=5$ in the right data set has been set to 35 - an outlier.

In [6]:


x4=x1.copy()
f_x=np.vectorize(lambda x: 3*x+2)(x4)
errors=np.random.normal(scale=4.0, size=x4.shape)
y4=f_x+errors
model_4=LinearRegressor()
model_4.fit(x4,y4)

x5=x4.copy()
y5=y4.copy()
y5[20]=35.0
model_5=LinearRegressor()
model_5.fit(x5,y5)


fig = make_subplots(rows=1, cols=2)
fig.add_trace(
    go.Scatter(x=x4, y=y4, mode="markers", name="Data set 4"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=x5, y=y5, mode="markers", name="Data set 5"),
    row=1, col=2
)
fig.add_shape(type="line",x0=0, y0=np.vdot(model_4.theta_f,np.array([0,1])), x1=10, y1=np.vdot(model_4.theta_f,np.array([10,1])), row=1, col=1)
fig.add_shape(type="line",x0=0, y0=np.vdot(model_5.theta_f,np.array([0,1])), x1=10, y1=np.vdot(model_5.theta_f,np.array([10,1])), row=1, col=2)
fig.show()
print(f"Theta_f for the left model: {model_4.theta_f}, Theta_f for the right model: {model_5.theta_f}")

Theta_f for the left model: [3.28177283 1.17227997], Theta_f for the right model: [3.25429272 1.78417049]


We can see that the slope of the two models is quite different - $1.46$ on the left and $2.01$ on the right. This problem becomes more extreme the fewer data points we have. Ideally, we would not want our model to be this sensitive to one outlier - we say that the *variance* of the model is high. One way of reducing the variance of a model is by increasing its *bias* - see variance bias  tradeoff for more details. We will do so using regularization.

**Unnecessary features** Consider the following data set, which is the same as the original data set, with added "dummy" feature that has no effect on y - the values in it are just Gaussian noise. Ideally, we would want our model to set the coefficient in $\theta_f$ to $0$, since it has no effect on the output. This is not what happens:

In [7]:
x6=x1.copy()
y6=y1.copy()
x6=np.stack([x6,np.random.normal(loc=5.0, scale=0.01, size=x6.shape)]).T
model_6=LinearRegressor()
model_6.fit(x6,y6)
print(f"{model_6.theta_f=}")

model_6.theta_f=array([  2.91989295,  15.83884735, -76.98972466])


We can see that the dummy feature gets a non zero coefficient. Using LASSO regularisation, we will be able to set such features to 0.


### Theory
To increase the bias of our regressor, we will add a panelty to the minimization problem that depends on the norm of $\theta_f$. We will start with **ridge regression** or L2 regression: denote by $\bar{\theta}$ the vecotr obtained from $\theta$ by dropping the last entry (i.e. the entry corresponding to the intercept). In ridge regression, instead of solving the problem $\min_\theta ||Y-X\theta||$, we solve:
$$
\min_\theta ||Y-X\theta||^2-\lambda||\bar{\theta}||^2
$$
where $\lambda$ is a constant. The larger $\lambda$ is, the larger the panelty for large slopes in $\theta_f$, and the larger the bias in the model - as $\lambda$ tends to infinity, the model will return lines that are closer to being horizontal.

Another type of regularisation is called LASSO (standing for Linear Absolute Shrinkage and Selection Operator) or L1 regression. Here, we solve:
$$
\min_\theta ||Y-X\theta||^2-\lambda||\bar{\theta}||_1
$$
where $||\cdot||_1$ is L1 norm. The main difference between ridge and LASSO regression is that ridge regression will never set the slope of a predictor to 0, whereas LASSO might do so. Thus, LASSO is very useful for filtering out useless features. On the other hand, ridge regression usually does better when most of the features are significant.

We can solve the ridge regression problem with simple vector calculus. Denote $L(\theta)=||Y-X\theta||^2-\lambda||\bar{\theta}||^2=(Y-X\theta)^t(Y-X\theta)-\lambda\theta^tA\theta$ where $A$ is the matrix obtained from the unit matrix by changing the 1 in the last row into a 0. We have:
$$
\frac{\partial L}{\partial \theta}=-2X^t(Y-X\theta)+2\lambda A\theta
$$
Equating this to 0, we get that the minimizing theta is:
$$
\theta_f=(X^tX+\lambda A)^{-1}X^tY
$$

For LASSO regression, the situation is more complicated. The L1 norm is not a differentiable function, so we can not find a closed solution using calculus. Instead, we will use gradient decent to find an approximate solution.

### Implementation
To implementing ridge regression, we only need to modify our linear regression class slightly.

In [8]:
class RidgeRegressor:
    def __init__(self, l2_penalty=1.0):
        self.theta_f=None
        self.penalty=l2_penalty
        
    def fit(self, X,y):
        if X.ndim==1:
            X1=np.stack([X,np.ones(shape=X.shape)]).T
        else:
            X1=np.vstack([X.T,np.ones(shape=X[:,0].shape)]).T
        A=np.identity(n=X1.shape[1])
        A[-1,-1]=0
        self.theta_f=np.linalg.inv(X1.T@X1+self.penalty*A)@X1.T@y
        
    def predict(self, X):
        if self.theta_f is None:
            raise Exception("The model has not been fit!")
        if X.ndim==1:
            temp=np.stack([X,np.ones(shape=X.shape)])
        else:
            temp=np.vstack([X.T,np.ones(shape=X[:,0].shape)])
        return self.theta_f@temp  

Ridge regression does particularly well when there are few data points, so we wil generate a new, small training set, and use a seprate testing set to assess the models accuracy. We will try several different penalties, and plot their MSEs on the test set.

In [9]:
x_train=np.array([[ 2, 2.59739473],
                 [4., 9.74957674],
                 [6., 10.75613676]])
x_test=np.array([[ 1.2, 2.252998  ],
                 [ 1.5,3.96387912],
                [ 1.8,5.810523  ],
                  [ 2.1, 3.64250821],
                  [ 2.4, 3.43019702],
                  [ 2.7,5.31171795],
                  [ 3., 8.57970934],
                  [ 3.3, 5.79632543],
                  [ 3.6, 8.83911681],
                  [ 3.9, 9.47770081],
                  [ 4.2, 7.84641176],
                  [ 4.5, 9.56898308],
                  [ 4.8,11.22839662],
                  [ 5.1,9.82087226],
                  [ 5.4,10.59641964],
                  [ 5.7,10.81831909]])
y_train=np.array([11.04059169, 18.29800831, 26.33709908,])
y_test=np.array([ 7.1070948,   8.28159273, 11.09492438, 11.10505032, 13.84774207, 13.12650938,
 15.27899416, 15.36465295, 19.54514913, 17.41240158, 20.10982071, 21.63377688,
 22.6137991,  23.21471234, 24.4701793,  25.84381147])

In [10]:
def compute_mse(y_true,y_pred):
    return np.linalg.norm(y_true-y_pred)/len(y_true)

mses=[]
for penalty in np.arange(0,10,0.2):
    model=RidgeRegressor(penalty)
    model.fit(x_train,y_train)
    y_pred=model.predict(x_test)
    mses.append(compute_mse(y_test,y_pred))
fig=px.scatter(x=np.arange(0,10,0.2),y=mses, title="MSEs of Ridge Regression Model", labels={"x":"penalty of model", "y":"MSE of model"})
#fig.add_shape(type="line", x0=0, y0=2, x1=10, y1=32)
fig.show()

We can see the MSE is minimized when the penalty is equal to 0.2.

For the LASSO regression, our model will require 2 more hyper parameters for the gradient decent: a learning rate and a number of itteration.

In [11]:
class LassoRegressor():
    def __init__(self, l1_penalty=1.0, learning_rate=0.01):
        self.theta_f=None
        self.penalty=l1_penalty
        self.learning_rate=learning_rate
        
    def fit(self, X, y, itterations):
        if X.ndim==1:
            X1=np.stack([X,np.ones(shape=X.shape)]).T
        else:
            X1=np.vstack([X.T,np.ones(shape=X[:,0].shape)]).T
        self.theta_f=np.zeros(shape=X1.shape[1])
        for _ in range(itterations):
            self.update(X,y)
            #print(f"{self.theta_f=}")
    
    def predict(self, X):
        if self.theta_f is None:
            raise Exception("The model has not been fit!")
        if X.ndim==1:
            temp=np.stack([X,np.ones(shape=X.shape)])
        else:
            temp=np.vstack([X.T,np.ones(shape=X[:,0].shape)])
        return self.theta_f@temp
    
    def update(self,X,y):
        if X.ndim==1:
            X=X.reshape(-1,1)
        y_pred=self.predict(X)
        grad=np.zeros(shape=self.theta_f.shape)
        for i in range(len(self.theta_f)-1):
            if self.theta_f[i]>0:
                grad[i] = (-2 * (X[:, i]).dot(y - y_pred) + self.penalty) / len(y)
            else:
                grad[i] = (-2 * (X[:, i]).dot(y - y_pred) - self.penalty) / len(y)
        grad[-1]=-2 * np.sum(y - y_pred) / len(y)
        #print(grad)
        self.theta_f-=self.learning_rate*grad


Lets see how the model does on the data set with the dummy feature:

In [12]:
x_train=np.arange(1,6,1)
f_x=np.vectorize(lambda x: 3*x+3)(x_train)
errors=np.random.normal(scale=1.0, size=x_train.shape)
y_train=f_x+errors
x_test=np.arange(1.2,6,0.3)
errors=np.random.normal(scale=1.0, size=x_test.shape)
y_test=np.vectorize(lambda x: 3*x**2+2*x+3)(x_test)+errors
for _ in range(5):
    x_train=np.c_[x_train,np.random.normal(scale=1.0, size=x_train.shape)]
    x_test=np.c_[x_test,np.random.normal(scale=1.0, size=x_test.shape)]
mses=[]
for penalty in np.arange(0,10,0.2):
    model=LassoRegressor(penalty, learning_rate=0.01)
    model.fit(x_train,y_train, 1000)
    y_pred=model.predict(x_test)
    mses.append(compute_mse(y_test,y_pred))
fig=px.scatter(x=np.arange(0,10,0.2),y=mses, title="MSEs of Lasso Regression Model", labels={"x":"penalty of model", "y":"MSE of model"})
#fig.add_shape(type="line", x0=0, y0=2, x1=10, y1=32)
fig.show()

Here the MSE is minimized when the penalty is equal to 1.2.

## Conclusion
Thanks for reading!